In [11]:
import django
import sys
import os
sys.path.append('..')

from django.conf import settings
django.setup()

from campfin_data.models import *
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
from django.db.models import Q, Sum, Count

In [12]:
expenses = ScrapedExpense.objects.all()
print expenses.count()
federal_refunds = expenses.filter(
    filing_committee__committee_group='federal', description__istartswith='refunds:')
print federal_refunds.count()
pdc_refunds = expenses.exclude(filing_committee__committee_group='federal').exclude(description__icontains='refundable').filter(description__icontains='refund')
print pdc_refunds.count()

79174
752
1319


In [13]:
print federal_refunds.aggregate(Sum('amount'))
descriptions = set([fr.description for fr in federal_refunds])
print descriptions

{'amount__sum': Decimal('216645.100')}
set([u'REFUNDS: REFUND OF EXCESS CONTIBUTION', u'REFUNDS: REFUND: EXCESS CONTRIBUTION', u'REFUNDS: REFUND OF CONTRIBUTION', u'REFUNDS: REFUND OF EXCESS CONTRIBUTON', u'REFUNDS: REFUND GENERAL ELECTION CONTRIBUTION', u'REFUNDS: REFUND', u'REFUNDS: REFUND TO DONOR', u'REFUNDS: CHECK RETURNED', u'REFUNDS: CONTRIBUTION REFUND', u'REFUNDS: REFUND: EXCESSIVE CONTRIBUTION', u'REFUNDS: REFUND: RETURNED CHECK - ACCOUNT CLOSED', u'REFUNDS: REFUND CONTRIBUTION'])


In [14]:
print pdc_refunds.aggregate(Sum('amount'))
descriptions = set([pr.description for pr in pdc_refunds])
print descriptions

{'amount__sum': Decimal('863702.910')}
set([u'REFUND', u"REFUND SEPT '15-MARCH '16 PAYROLL DEDUCTIONS TO WEA-PAC SENT IN ERROR", u"REFUND DEC. '15 PAYROLL DEDUCTION TO WEA-PAC. WEA MEMBERSHIP CANCELLED BY INDIVIDUAL", u'REFUND PRIMARY CONTRIBUTION', u'REFUND 2013 CONTRIBUTION', u"REFUND MAY '16 PAYROLL DEDUCTION OVERPAYMENT TO WEA-PAC MADE IN ERROR.", u'REFUND FOR CONTRIBUTION EXCEEDING $1, 000', u'REFUND OF CONTRIBUTION FROM 2015', u'REFUND 2015 PAYROLL DEDUCTION TO WEA-PAC SENT IN ERROR', u'REFUND AUG 16 PAYROLL DEDUCTION TO WEA-PAC PER MEMBER REQUEST', u"REFUND DEC '15-FEB '16 PAYROLL DEDUCTIONS TO WEA-PAC. AGENCY FEE PAYORS INELIGIBLE TO CONT", u"REFUND OCT-DEC '15 PAYROLL DEDUCTIONS TO WEA-PAC. NO AUTHORIZATION FORM ON FILE W/OTHELLO", u'REFUND OF LDD TICKET', u'REFUND OF DISTRICT CAUCUS FEE CHARGED TWICE', u'REFUND DUE TO GENERAL LIMIT EXCEEDED', u'REFUND CASH CONTRIBUTION', u"REFUND...DIDN'T MEAN TO DONATE $100", u'2015 ANNUAL DINNER TICKET REFUND', u'REFUND FEB-MARCH 16 PAYROLL

In [15]:
federal_campaigns = federal_refunds.values('filing_committee__name').annotate(sum=Sum('amount')).order_by('-sum')
for fc in federal_campaigns:
    print '%s: %s' % (fc['filing_committee__name'], locale.currency(fc['sum'], grouping=True))

CLINTON, HILLARY RODHAM / KAINE, TIMOT: $80,332.31
MURRAY, PATTY: $56,639.79
TRUMP, DONALD J. / MICHAEL R. PENCE : $21,025.47
MCDERMOTT, JAMES JOSEPH: $18,601.55
MCMORRIS RODGERS, CATHY: $12,680.00
MCDERMOTT, JAMES: $10,700.00
JAYAPAL, PRAMILA: $6,450.00
DELBENE, SUZAN K: $5,522.38
HERRERA BEUTLER, JAIME: $3,600.00
KAY, DAVID: $723.60
VENTRELLA, TONY: $270.00
WALKINSHAW, BRADY PINERO: $100.00


In [16]:
pdc_campaigns = pdc_refunds.values('filing_committee__name').annotate(sum=Sum('amount')).order_by('-sum')
for pc in pdc_campaigns:
    print '%s: %s' % (pc['filing_committee__name'], locale.currency(pc['sum'], grouping=True))

SAVE ANIMALS FACING EXTINCTION: $187,969.68
GOLDMARK PETER J: $114,483.59
RAISE UP WA: $90,000.00
ROSEN STEVEN G: $43,995.00
WILSON LYNDA D: $42,220.16
MCHALE JOHN: $39,111.00
ROACH DANIEL N: $24,242.00
INSLEE JAY R: $24,023.21
VOTERS FOR WASHINGTON CHILDREN: $21,447.79
SCHOESLER MARK G: $20,025.00
YES FOR HOMES: $17,500.00
HOBBS STEVEN R: $15,890.00
FIELD IAN A: $13,000.00
NO ON I-123: $12,386.25
CARLSON DYLAN: $12,131.00
PORTERFIELD KAREN M: $12,000.00
UPTHEGROVE DAVID P: $10,000.00
FERGUSON ROBERT W: $9,150.00
LIIAS MARKO S: $8,000.00
WA ST DEMO CENT COMM NON EXEMPT: $7,590.00
SULLIVAN PATRICK J: $6,650.00
HABIB CYRUS: $5,550.00
KENNEDY FUND: $5,250.00
MCINTIRE JAMES L: $4,800.00
SPRUNG JEFFREY T: $4,800.00
OLSON BRADY J: $4,635.00
BECKER RANDI L: $4,200.00
MCAULIFFE ROSEMARY A: $4,150.00
TURNER TIFFANY A: $4,000.00
CONNECTING CLARK CO: $3,784.87
ALLIANCE FOR GUN RESPONSIBILITY ERP COMM: $3,525.00
HUMANE VOTERS OF WA: $3,500.00
ZEIGER HANS A: $3,400.00
MOSCOSO LUIS S: $3,300.00
PROB

In [17]:
for refund in pdc_refunds.filter(filing_committee__name='INSLEE JAY R').order_by('-amount'):
    print refund.vendor, refund.amount, refund.description

XIAO MING 2500.000 REFUND
POWER PHILIP 2080.000 REFUND
REED RONALD 2000.000 REFUND
THOMPSON ROWLAND 2000.000 REFUND
UNION PACIFIC RAILROAD 2000.000 CONTRIBUTION REFUND
CEDAR GROVE COMPOSTING INC 1900.000 REFUND OVERLIMIT CONTRIBUTION
WATERSIDE ENERGY 1900.000 REFUND
MILLENNIUM PHARMACEUTICALS PAC 1800.000 REFUND
THOMPSON ROWLAND 1750.000 REFUND
DAUDON MAUD S 1250.000 REFUND
YOUNG JIM D. 650.000 REFUND
BROOKINS CHERYLL 500.000 REFUND
EDP RENEWABLES NORTH AMERICA LLC PAC 500.000 REFUND CONTRIBUTION
GARRENS DEBBIE 500.000 REFUND
JACOBSON STUART L 500.000 REFUND
MCGINNIS KAREN 250.000 REFUND
ROSEN DEBORAH M. 250.000 REFUND
SANDUSKY WILLIAM F. 250.000 REFUND
WEN SHEREE 250.000 REFUND
COATS HERBERT 242.000 REFUND
LAMB-GUNNERSON COLLEEN 200.000 REFUND
KAGI RUTH L 176.210 REFUND
CAIN BARRY A 150.000 REFUND OVERLIMIT CONTRIBUTION
ASHLEY LAURENCE 125.000 REFUND
MCCRUMMEN LISA 100.000 CONTRIBUTION REFUND
TATE SHARON D 100.000 REFUND OVERLIMIT CONTRIBUTION
VREELAND VICTORIA L. 100.000 CONTRIBUTION

In [18]:
for refund in pdc_refunds.values('vendor', 'city', 'state').annotate(sum=Sum('amount')).order_by('-sum'):
    print refund['vendor'], refund['sum']

ALLEN PAUL 187969.680
SEIU WASHINGTON STATE COUNCIL 90000.000
17TH LEGISLATIVE DISTRICT REPUBLICAN COMMITTEE 39000.000
WASHINGTON CHARTERS PAC 14298.530
FRIENDS OF SEATTLE WATERFRONT 12386.250
LOW INCOME HOUSING INSTITUTE 10000.000
ALAN & KRISTIN ROSEN 8000.000
FREDERICK & GAIL HINES 8000.000
STAND FOR CHILDREN WASHINGTON PAC 7149.260
WASHINGTON HEALTH CARE ASSOCIATION PAC 5619.460
LUMMI INDIAN BUSINESS COUNCIL 5396.000
BECU 5000.000
THOMPSON ROWLAND 5000.000
PLUMBERS & STEAMFITTERS LOCAL UNION 598 PAC 5000.000
POSEROW BEVERLY 4000.000
MACDONALD ALEXANDER 4000.000
ROSEN GEOFFREY 4000.000
ROSEN MARVIN 4000.000
TAYEBI MASSIH 3800.000
LONGBRAKE WILLIAM A 3800.000
BRYAN & CHRISTINE WHITE 3800.000
SWINOMISH TRIBAL COMMUNITY 3546.000
SMITH SANDY 3500.000
TULALIP TRIBES OF WASHINGTON 3496.000
YOUNG'S MARKET COMPANY 3300.000
NORDSTROM BRUCE A 3146.000
PUGET SOUND PILOTS PAC 3000.000
LUMMI COMMERCIAL COMPANY 2950.000
WASHINGTON BEVERAGE ASSOCIATION POLITICAL ACTION COMMITTEE 2950.000
SEIU 775 Q

In [19]:
for refund in federal_refunds.values('vendor', 'city', 'state').annotate(sum=Sum('amount')).order_by('-sum'):
    print refund['vendor'], refund['sum']

TREIBEL, RANDY 14440.030
SUSSMAN, S DONALD 5400.000
INGRAM CAUCHI, ALEXA ANDREA 4700.000
CONFEDERATED TRIBES OF THE CHEHALIS RESERVATION 4100.000
HILLS, JAMIE 4050.000
RHODES, CARRIE 3450.000
RICKERT, ALLEN 3112.500
ALSAKER, ANNE 2800.000
KUNEY, MAX 2750.000
KERR, ROBERT 2700.750
TAGNEY-JONES, MARYANNE 2700.000
RAIKES, PATRICIA M 2700.000
JONES, DAVID 2700.000
BUCKLEY, JODY 2700.000
RATTLINGOURD, GLEN 2700.000
TAGNEY, MARYANNE 2700.000
HOROWITZ, MARK 2700.000
BJORNSTAD, JEFF 2700.000
ARNOLD, LEOLA 2700.000
FARRIS, JULI E. 2700.000
FEENEY, SUSAN 2600.000
RAIKES, JEFFREY 2600.000
GIULIANI, DAVID 2500.000
JOHNSON, LINDA 2400.000
DEGEL, JAMES A. 2400.000
SHAW, GREGORY L 2300.000
NORDSTROM, DANIEL 2300.000
BETHNA CULPEPPER 2300.000
BARER, STANLEY H. 2300.000
HAUENSTEIN, RONALD E 2240.000
HUPPIN, MURRAY 2240.000
ALSAKER, DANIEL L 2200.000
CHRIST, GRACE HYLSOP 2182.000
ROBINSON, JEFFREY 2100.000
KEARNEY, LEE 2000.000
KIM, CHANG 2000.000
BARRY, LOUIS 2000.000
KOZA, JOHN R DR 1859.560
SINEGAL, 

In [20]:
for refund in federal_refunds.filter(filing_committee__name='MURRAY, PATTY').order_by('-amount'):
    print refund.vendor, refund.amount, refund.description

HILLS, JAMIE 4050.000 REFUNDS: CONTRIBUTION REFUND
RICKERT, ALLEN 3112.500 REFUNDS: CONTRIBUTION REFUND
RAIKES, PATRICIA M 2700.000 REFUNDS: CONTRIBUTION REFUND
RAIKES, JEFFREY 2600.000 REFUNDS: CONTRIBUTION REFUND
GIULIANI, DAVID 2500.000 REFUNDS: CONTRIBUTION REFUND
CONFEDERATED TRIBES OF THE CHEHALIS RESERVATION 2500.000 REFUNDS: CONTRIBUTION REFUND
JOHNSON, LINDA 2400.000 REFUNDS: CONTRIBUTION REFUND
SHAW, GREGORY L 2300.000 REFUNDS: CONTRIBUTION REFUND
NORDSTROM, DANIEL 2300.000 REFUNDS: CONTRIBUTION REFUND
ROBINSON, JEFFREY 2100.000 REFUNDS: CONTRIBUTION REFUND
KOZA, JOHN R DR 1859.560 REFUNDS: CONTRIBUTION REFUND
HUNTER, SUZANNE 1800.000 REFUNDS: CONTRIBUTION REFUND
CONFEDERATED TRIBES OF THE CHEHALIS RESERVATION 1600.000 REFUNDS: CONTRIBUTION REFUND
BOCHCO, DAYNA MRS 1500.000 REFUNDS: CONTRIBUTION REFUND
WARNER, RICHARD 1300.000 REFUNDS: CONTRIBUTION REFUND
ANDERSON, ERIC 1150.000 REFUNDS: CONTRIBUTION REFUND
LARUE, ANDREA 1000.000 REFUNDS: CONTRIBUTION REFUND
ROBERTS, JOANNE M

In [21]:
for refund in pdc_refunds.filter(vendor='LOW INCOME HOUSING INSTITUTE'):
    print refund.vendor, refund.amount, refund.filing_committee.name

LOW INCOME HOUSING INSTITUTE 10000.000 YES FOR HOMES
